In [0]:
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential, model_from_json
from keras.utils import to_categorical
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers import MaxPooling2D
from keras.optimizers import RMSprop
from keras.layers.convolutional import Conv2D
from keras.callbacks import ModelCheckpoint
from keras.layers import Embedding, TimeDistributed, RepeatVector, LSTM, concatenate , Input, Reshape, Dense,GRU
from keras.preprocessing.image import array_to_img, img_to_array, load_img
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
#load dataset and organize for easy parsing
#DONE: train1
#helper function: read file and return txt as a string
def load_doc(filename):
    with open(filename,'r') as file:
      text = file.read()
      file.close()
    return text

def load_data():
    text = []
    images = []
    #load all files + sort them
    all_filenames = listdir("/content/drive/My Drive/Colab Notebooks/ScreenshotTest/train/train16")
    all_filenames.sort()
    for filename in (all_filenames):
        if filename[-3:]=="npz":
            #load imgs already prepped in arr
            image = np.load("/content/drive/My Drive/Colab Notebooks/ScreenshotTest/train/train16/"+filename)
            images.append(image['features'])
        else:
            #load bootstrap tokens LABELS/TARGETS
            syntax = '<START>' + load_doc("/content/drive/My Drive/Colab Notebooks/ScreenshotTest/train/train16/"+filename) + '<END'
            syntax = ' '.join(syntax.split())
            syntax = syntax.replace(',',' ,')
            text.append(syntax)
            
    images = np.array(images,dtype=float)
   # image_File = np.save('/content/drive/My Drive/Colab Notebooks/ScreenshotTest/image_file',images)
   # text_file = np.save('/content/drive/My Drive/Colab Notebooks/ScreenshotTest/text_fie',text)
    return images,text
  
train_features, texts = load_data()

In [0]:
tokenizer = Tokenizer(filters='',split=" ",lower=False)
tokenizer.fit_on_texts([load_doc('/content/drive/My Drive/Colab Notebooks/ScreenshotTest/resources/bootstrap.vocab')])

vocab_size = len(tokenizer.word_index)+1
#map input sentences into vocabulary indexes
train_sequences = tokenizer.texts_to_sequences(texts)
#longest set of bootstrap tok
max_sequence = max(len(s) for s in train_sequences)
max_length = 48


def preprocess_data(sequences, features):
    X, y, image_data = list(), list(), list()
    for img_no, seq in enumerate(sequences):
        for i in range(1, len(seq)):
            # Add the sentence until the current count(i) and add the current count to the output
            in_seq, out_seq = seq[:i], seq[i]
            # Pad all the input token sentences to max_sequence
            in_seq = pad_sequences([in_seq], maxlen=max_sequence)[0]
            # Turn the output into one-hot encoding
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            # Add the corresponding image to the boostrap token file
            image_data.append(features[img_no])
            # Cap the input sentence to 48 tokens and add it
            X.append(in_seq[-48:])
            y.append(out_seq)
    return np.array(X), np.array(y), np.array(image_data)

X, y, image_data = preprocess_data(train_sequences, train_features)

In [0]:
#Create the encoder
#image_model = Sequential()
#image_model.add(Conv2D(16, (3, 3), padding='valid', activation='relu', input_shape=(256, 256, 3,)))
#image_model.add(Conv2D(16, (3,3), activation='relu', padding='same', strides=2))
#image_model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
#image_model.add(Conv2D(32, (3,3), activation='relu', padding='same', strides=2))
#image_model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
#image_model.add(Conv2D(64, (3,3), activation='relu', padding='same', strides=2))
#image_model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
#image_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

#image_model.add(Flatten())
#image_model.add(Dense(1024, activation='relu'))
#image_model.add(Dropout(0.3))
#image_model.add(Dense(512, activation='relu'))
#image_model.add(Dropout(0.3))

#image_model.add(RepeatVector(max_length))

#visual_input = Input(shape=(256, 256, 3,))
#encoded_image = image_model(visual_input)

#language_input = Input(shape=(max_length,))
#language_model = Embedding(vocab_size, 50, input_length=max_length, mask_zero=True)(language_input)
#language_model = GRU(128, return_sequences=True)(language_model)
#language_model = GRU(128, return_sequences=True)(language_model)

#Create the decoder
#decoder = concatenate([encoded_image, language_model])
#decoder = GRU(512, return_sequences=True)(decoder)
#decoder = GRU(512, return_sequences=False)(decoder)
#decoder = Dense(vocab_size, activation='softmax')(decoder)

#LOAD MODEL FROM JSON
with open('/content/drive/My Drive/Colab Notebooks/ScreenshotTest/modelGRU.json','r') as json_file:
  model_json = json_file.read()
  json_file.close()
model = model_from_json(model_json)
model.load_weights("/content/drive/My Drive/Colab Notebooks/ScreenshotTest/weightsGRU.h5")
print("Loaded model from disk")







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






Loaded model from disk


In [0]:
# Compile the model
#model = Model(inputs=[visual_input, language_input], outputs=decoder)

optimizer = RMSprop(lr=0.0001, clipvalue=1.0)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['categorical_accuracy'])
#model.summary()
                

In [0]:
#save data
import pickle as pickle
epoch_file = '/content/drive/My Drive/Colab Notebooks/ScreenshotTest/epochsDataMyModel28.txt'
data_file='/content/drive/My Drive/Colab Notebooks/ScreenshotTest/data1CategoricalMyModel28.txt'

report_data = {
    "accuracy":[],
    "val_accuracy":[],
    "loss":[],
    "val_loss":[]
}
num_epoch =0

def updateEpoch(epoch, logs):
  to_save = num_epoch + epoch + 1
  report_data['accuracy'].append(logs['categorical_accuracy'])
  report_data['loss'].append(logs['loss'])
  report_data['val_accuracy'].append(logs['val_categorical_accuracy'])
  report_data['val_loss'].append(logs['val_loss'])
  with open(epoch_file, "w") as file:  
    file.write(str(to_save))
  with open(data_file, "wb") as file:
    pickle.dump(report_data, file)
  #print(epoch, logs)


In [0]:
from keras.callbacks import LambdaCallback
#Save the model for every 2nd epoch
filepath="/content/drive/My Drive/Colab Notebooks/ScreenshotTest/train/train1/weightstrainEDITGRU.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_weights_only=True, period=2)
lambdaCall = LambdaCallback(on_epoch_end=updateEpoch)
callbacks_list = [checkpoint,lambdaCall]

In [0]:
# Train the model
model.fit([image_data, X], y, batch_size=32, shuffle=False, validation_split=0.1, callbacks=callbacks_list, verbose=1, epochs=40)



Train on 3107 samples, validate on 346 samples
Epoch 1/40
3107/3107 [==============================] - 70s 22ms/step - loss: 0.3920 - categorical_accuracy: 0.9292 - val_loss: 0.3703 - val_categorical_accuracy: 0.9364
Epoch 2/40
3107/3107 [==============================] - 64s 20ms/step - loss: 0.1956 - categorical_accuracy: 0.9453 - val_loss: 0.2561 - val_categorical_accuracy: 0.9422

Epoch 00002: saving model to /content/drive/My Drive/Colab Notebooks/ScreenshotTest/train/train1/weightstrainEDITGRU.best.hdf5
Epoch 3/40
3107/3107 [==============================] - 64s 21ms/step - loss: 0.1287 - categorical_accuracy: 0.9611 - val_loss: 0.2602 - val_categorical_accuracy: 0.9422
Epoch 4/40
3107/3107 [==============================] - 63s 20ms/step - loss: 0.0930 - categorical_accuracy: 0.9691 - val_loss: 0.2641 - val_categorical_accuracy: 0.9422

Epoch 00004: saving model to /content/drive/My Drive/Colab Notebooks/ScreenshotTest/train/train1/weightstrainEDITGRU.best.hdf5
Epoch 5/40
3107

In [0]:
model_json = model.to_json()
with open("/content/drive/My Drive/Colab Notebooks/ScreenshotTest/modelGRU.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/drive/My Drive/Colab Notebooks/ScreenshotTest/weightsGRU.h5")
print("Saved model to disk")

In [0]:
print("Done! 6:39PM")

In [0]:
print("Done! ")

In [0]:
#summarize for accuracy
plt.plot(report_data['accuracy'])
plt.plot(report_data['val_accuracy'])
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('iterations')
plt.legend(['training','target'],loc='upper left')
plt.show()

#summarize for errors
plt.plot(report_data['loss'])
plt.plot(report_data['val_loss'])
plt.ylabel('loss')
plt.xlabel('iterations')
plt.title('Learning Rate')
plt.legend(['training','target'], loc='upper left')
plt.show()

In [0]:
print("done")